In [119]:
! py -m pip install "numpy<2"

  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
langchain-experimental 0.0.57 requires langchain<0.2.0,>=0.1.15, but you have langchain 0.2.11 which is incompatible.
langchain-experimental 0.0.57 requires langchain-core<0.2.0,>=0.1.41, but you have langchain-core 0.2.24 which is incompatible.
llama-index-embeddings-openai 0.1.11 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.20 which is incompatible.
llama-index-indices-managed-llama-cloud 0.1.6 requires llama-index-core<0.11.0,>=0.10.0, but you have llama-index-core 0.11.20 which is incompatible.
llama-in

In [1]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import pandas as pd
import os
import shutil
import numpy as np
import re
import sqlite3
from langchain_community.vectorstores.faiss import FAISS

In [2]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://az-openai-document-question-answer-service.openai.azure.com/" 
OPENAI_API_KEY = "5d24331966b648738e5003caad552df8" 
OPENAI_API_VERSION = "2023-05-15"

OPENAI_DEPLOYMENT_NAME = "az-gpt_35_model"
OPENAI_MODEL_NAME="gpt-3.5-turbo"

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = "az-embedding_model" 
OPENAI_ADA_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

encoding_name = "cl100k_base"

csv_file = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\Claims\\vm-GenAI_BI\\Text to SQL_V1\\data\\agg_claim_data.csv"
### Cohere API key 

In [3]:
llm = AzureChatOpenAI(
                        temperature=0.1,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

embedding_model = AzureOpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,
                                            deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
                                            model=OPENAI_ADA_EMBEDDING_MODEL_NAME,
                                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT)

def get_embedding():
    return embedding_model
# data = load_documents()

In [4]:
# Load CSV into SQLite
def load_csv_to_sqlite(csv_file, db_file, table_name):
    conn = sqlite3.connect(db_file)
    df = pd.read_csv(csv_file, encoding='latin-1')
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Data from {csv_file} loaded into SQLite table {table_name}.")

In [ ]:
import pandas as pd

# Load data

df = pd.read_csv(csv_file)

# Get unique loss_cause values
unique_loss_causes = df['loss_cause'].dropna().unique().tolist()
print("Unique loss causes:", unique_loss_causes)


Unique loss causes: ['Animal Collision', 'All Other Property Losses', 'WATER BACK-UP OF SEWERS & DRAINS', 'Wind', 'ANIMAL COLLISION', 'Glass breakage', 'FIRE', 'Falling or moving object', 'ALL OTHERS', 'CANINE/FELINE', 'WIND', 'Hail excluding losses only to roof', 'Hail Loss only to roof', 'HAIL', 'WATER DAMAGE', 'MYSTERIOUS DISAPPERANCEE - ON PREMISES', 'Hail', 'THEFT ON PREMISES', 'Collision with motor vehicle', 'TOTAL THEFT', 'ALL OTHER', 'All Other', 'VEHICLES OR AIRCRAFT', 'Tire Loss', 'DOG BITES', 'Water Back-up of Sewers & Drains', 'Water and Sewer Line', 'Windstorm', 'GLASS', 'Malicious mischief and vandalism', 'Water Damage', 'Fireplaces', 'MEDICAL PAYMENTS', 'VANDALISM', 'All Other Liability Losses', 'LIGHTNING', 'FLOOD', 'WATER DAMAGE (EXCLUDING FLOOD)', 'Medical Payments', 'WINDSTORM', 'WATERCRAFT - ALL OTHER (BOATOWNERS ONLY)', 'Lightning', 'MOLD - WET/DRY ROT', 'Theft On Premises', 'FREEZING, ICE DAM, COLLAPSE DUE TO WEIGHT OF ICE AND SNOW', 'BREAKAGE', 'Theft of entire v

In [8]:
from langchain.embeddings import AzureOpenAIEmbeddings


# Generate embeddings for unique loss causes
loss_cause_embeddings = embedding_model.embed_documents(unique_loss_causes)

faiss_store = FAISS.from_texts(
    texts=unique_loss_causes,
    embedding=embedding_model
)

print("FAISS index created for loss causes!")



FAISS index created for loss causes!


In [10]:
# User query
user_query = "glass breakage"

# Perform similarity search
matches = faiss_store.similarity_search(user_query, k=1)

# Get the closest match
if matches:
    closest_match = matches[0].page_content  # Retrieve the matched content
    print(f"Closest match for '{user_query}': {closest_match}")
else:
    print("No close match found.")


Closest match for 'glass breakage': Glass breakage


In [14]:
db_file = "claims.db"
table_name = "claims"

In [15]:
load_csv_to_sqlite(csv_file, db_file, table_name)

Data from C:\Users\SujaySunilNagvekar\VM\GEN AI\Claims\vm-GenAI_BI\Text to SQL_V1\data\agg_claim_data.csv loaded into SQLite table claims.


In [16]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Reconnect to the SQLite database
conn = sqlite3.connect("claims.db")

# Query to fetch the top 10 rows from the Claims table
query = "SELECT * FROM claims LIMIT 10"

# Use pandas to execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Print the DataFrame as a formatted table
print(tabulate(df, headers='keys', tablefmt='grid'))

# Close the connection
conn.close()


+----+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------+---------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------+----------------+--------------------+-------------------+---------------------------+------------+-------------+-----------+-------------------+--------------------+---------------+---------------+--------------+--------------+--------------------------+-----------+-----------------+----------+----------------+--------------------+------------

In [17]:
def generate_table_schema():
    schema = [
        {"Column Name": "claim_no", "Data Type": "VARCHAR", "Null": "NO", "Primary Key": "YES"},
        {"Column Name": "fnol_call", "Data Type": "VARCHAR", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "LossDate", "Data Type": "DATE", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "loss_cause", "Data Type": "VARCHAR", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "loss_location_zip", "Data Type": "VARCHAR", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "claim_description", "Data Type": "TEXT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "LOB", "Data Type": "VARCHAR", "Null": "NO", "Primary Key": "NO"},
        {"Column Name": "PolicyNumber", "Data Type": "VARCHAR", "Null": "NO", "Primary Key": "NO"},
        {"Column Name": "direct_paid_loss", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "direct_paid_LAE", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "direct_outstanding_loss", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "CAT_flag", "Data Type": "BOOLEAN", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "loss_year", "Data Type": "INTEGER", "Null": "NO", "Primary Key": "NO"},
        {"Column Name": "Age_max", "Data Type": "INTEGER", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "CoverageA_Limit", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "ConstructionYear", "Data Type": "INTEGER", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "DriverCount", "Data Type": "INTEGER", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "CreditScore", "Data Type": "INTEGER", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "bi_limit_1", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "bi_limit_2", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "one_auto_full_cov_flag", "Data Type": "BOOLEAN", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "Age_min", "Data Type": "INTEGER", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "MaritalStatus", "Data Type": "VARCHAR", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "Gender", "Data Type": "VARCHAR", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "VehicleCount", "Data Type": "INTEGER", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "AgencyProductNam", "Data Type": "VARCHAR", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "EarnedPremium", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "AnnualizedIPAmt", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "minPolicyEffectiveDt", "Data Type": "DATE", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "minPolicyInceptionDts", "Data Type": "DATE", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "maxPolicyExpirationDt", "Data Type": "DATE", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "maxPolicyCancelDt", "Data Type": "DATE", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "IncurredLossCat", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "IncurredLossNonCat", "Data Type": "FLOAT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "adjuster_notes", "Data Type": "TEXT", "Null": "YES", "Primary Key": "NO"},
        {"Column Name": "num_adjuster_notes", "Data Type": "INTEGER", "Null": "YES", "Primary Key": "NO"}
    ]
    
    # Format schema as a string for LLM input
    formatted_schema = "\n".join(
        f"Column Name: {col['Column Name']}, Data Type: {col['Data Type']}, Null: {col['Null']}, Primary Key: {col['Primary Key']}"
        for col in schema
    )
    return formatted_schema

# Example usage


In [18]:
def create_table_metadata():
    metadata = [
        {"Column Name": "claim_no", "Description": "Unique identifier for the insurance claim."},
        {"Column Name": "fnol_call", "Description": "First Notice of Loss call identifier or details."},
        {"Column Name": "LossDate", "Description": "Date of the reported loss."},
        {"Column Name": "loss_cause", "Description": "Cause of the loss (e.g., fire, theft)."},
        {"Column Name": "loss_location_zip", "Description": "ZIP code where the loss occurred."},
        {"Column Name": "claim_description", "Description": "Detailed description of the claim."},
        {"Column Name": "LOB", "Description": "Line of Business associated with the claim."},
        {"Column Name": "PolicyNumber", "Description": "Policy number linked to the claim."},
        {"Column Name": "direct_paid_loss", "Description": "Amount directly paid for the loss."},
        {"Column Name": "direct_paid_LAE", "Description": "Amount directly paid for Loss Adjustment Expenses."},
        {"Column Name": "direct_outstanding_loss", "Description": "Outstanding amount for the loss."},
        {"Column Name": "CAT_flag", "Description": "Indicates if the claim is related to a catastrophe event."},
        {"Column Name": "loss_year", "Description": "Year when the loss occurred."},
        {"Column Name": "Age_max", "Description": "Maximum age of the insured person(s)."},
        {"Column Name": "CoverageA_Limit", "Description": "Limit of Coverage A for the policy."},
        {"Column Name": "ConstructionYear", "Description": "Year the insured property was constructed."},
        {"Column Name": "DriverCount", "Description": "Number of drivers covered under the policy."},
        {"Column Name": "CreditScore", "Description": "Credit score of the insured person or entity."},
        {"Column Name": "bi_limit_1", "Description": "First limit for bodily injury coverage."},
        {"Column Name": "bi_limit_2", "Description": "Second limit for bodily injury coverage."},
        {"Column Name": "one_auto_full_cov_flag", "Description": "Indicates if one auto has full coverage."},
        {"Column Name": "Age_min", "Description": "Minimum age of the insured person(s)."},
        {"Column Name": "MaritalStatus", "Description": "Marital status of the insured person."},
        {"Column Name": "Gender", "Description": "Gender of the insured person."},
        {"Column Name": "VehicleCount", "Description": "Number of vehicles covered under the policy."},
        {"Column Name": "AgencyProductNam", "Description": "Name of the agency product linked to the policy."},
        {"Column Name": "EarnedPremium", "Description": "Earned premium amount for the policy."},
        {"Column Name": "AnnualizedIPAmt", "Description": "Annualized installment premium amount."},
        {"Column Name": "minPolicyEffectiveDt", "Description": "Earliest effective date of the policy."},
        {"Column Name": "minPolicyInceptionDts", "Description": "Earliest inception date of the policy."},
        {"Column Name": "maxPolicyExpirationDt", "Description": "Latest expiration date of the policy."},
        {"Column Name": "maxPolicyCancelDt", "Description": "Latest cancellation date of the policy."},
        {"Column Name": "IncurredLossCat", "Description": "Incurred loss amount for catastrophe claims."},
        {"Column Name": "IncurredLossNonCat", "Description": "Incurred loss amount for non-catastrophe claims."},
        {"Column Name": "adjuster_notes", "Description": "Notes or comments added by the claim adjuster."},
        {"Column Name": "num_adjuster_notes", "Description": "Number of notes made by the adjuster."}
    ]
    
    # Format metadata as a string for LLM input
    formatted_metadata = "\n".join(
        f"Column Name: {col['Column Name']}, Description: {col['Description']}" for col in metadata
    )
    return formatted_metadata

# Example usage
metadata_text = create_table_metadata()


In [19]:
table_schema = generate_table_schema()
table_metadata = create_table_metadata()

In [20]:
class ConversationManager:
    def __init__(self, history_limit=5):
        """
        Initialize the conversation manager with a history limit.
        """
        self.history = []
        self.history_limit = history_limit

    def update_conversation_history(self, question, answer):
        """
        Store previous questions and answers, but limit the history size.
        """
        self.history.append(f"Question: {question} \n Answer: {answer}")
        if len(self.history) > self.history_limit:
            self.history.pop(0)  # Remove the oldest entry to keep the history within the limit

    def get_conversation_context(self):
        """
        Get the full conversation context as a string.
        """
        return self.history
    
    def clear_history(self):
        """
        Clear the conversation history to erase context.
        """
        self.history = []

    def reformulate_question(self, query_text, llm):
        """
        Reformulate the follow-up question based on the conversation history.
        """
               # Use the reformulation prompt to generate a standalone question
        reformulation_prompt = """
        Given a chat history ( User Questions and LLM response ) and the latest user question which might reference context in the chat history, 
        formulate a standalone question which can be understood without the chat history. Do NOT answer the question, 
        just reformulate it if needed and otherwise return it as is.

        Chat History:
        {chat_history}

        Latest Question:
        {latest_question}
        """

        # Prepare the input for the LLM model
        formatted_prompt = reformulation_prompt.format(
            chat_history=self.get_conversation_context(),
            latest_question=query_text
        )

        # Get the reformulated question from LLM
        reformulated_response = llm.invoke(input=formatted_prompt)
        standalone_question = reformulated_response.content.strip()

        return standalone_question

In [153]:
def preprocess_sql_query(sql_query, faiss_store, embedding_model):
    """
    Standardize loss_cause in the SQL query using the FAISS vector store.
    
    :param sql_query: The SQL query generated by the LLM
    :param faiss_store: The FAISS vector store containing loss_cause embeddings
    :param embedding_model: The embedding model to create embeddings for user input
    :return: The updated SQL query with standardized loss_cause
    """
    import re

    # Check if "loss_cause" is in the SQL query
    if "loss_cause" in sql_query.lower():
        # Extract the loss_cause value from the WHERE clause
        match = re.search(r"loss_cause\s*(ILIKE|LIKE|=)\s*'%([^']+)%?'", sql_query, re.IGNORECASE)
        if match:
            loss_cause_value = match.group(2)

            # Query the FAISS vector store for the closest match
            query_embedding = embedding_model.embed_query(loss_cause_value)
            matches = faiss_store.similarity_search_by_vector(query_embedding, k=1)

            if matches:
                closest_match = matches[0].page_content  # Get the closest standardized value
                print(f"Standardized loss_cause: {closest_match}")

                # Replace the original value with the standardized value
                # Ensure '%' wildcards are preserved for LIKE/ILIKE clauses
                updated_value = f"%{closest_match}%" if "LIKE" in match.group(1).upper() else closest_match
                sql_query = sql_query.replace(f"'{loss_cause_value}%?'", f"'{updated_value}'")
            else:
                print("No close match found in FAISS for loss_cause.")
        else:
            print("No loss_cause value found in the query.")
    return sql_query


In [154]:
# Execute the SQL query
def execute_sql_query_with_preprocessing(db_file, sql_query, faiss_store, embedding_model):
    # Preprocess the SQL query for loss_cause
    print(f"Original SQL query: {sql_query}")
    standardized_sql_query = preprocess_sql_query(sql_query, faiss_store, embedding_model)
    print(f"standard SQL query:{standardized_sql_query}")
    # Connect to the database and execute the query
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    try:
        cursor.execute(standardized_sql_query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()
        return pd.DataFrame(result, columns=columns)
    except sqlite3.Error as e:
        conn.close()
        return f"SQL error: {e}"


In [155]:
# Generate SQL Query using AzureChatOpenAI LLM
def generate_sql_query(llm, natural_language_query, table_metadata, table_name, schema):
    query_prompt = f"""
You are a SQL expert. Use the table schema and metadata provided below to generate a SQL query. The data contains information about Insurance Claims, FNOL call, Adjustor notes and other details. 

<table info>

Table: {table_name}
Schema:
{schema}
Metadata: 
{table_metadata}

<table info/>

<Special Instructions>
- The SQL query must extract data semantically from text-based columns like 'adjuster_notes' , 'fnol_call'
without relying on exact pattern matching (e.g., LIKE or CONTAINS).
- Summarize or Explain means to read the entire Unstructured / Text Column requested
- claim no. , Claim Number , claim # - all mean the same primary key column

<Special Instructions/>

Convert the following natural language query into a SQL query:

{natural_language_query}

Only return the SQL query, no additional text.
"""
    response = llm.invoke(query_prompt)
    return response.content

# Execute the SQL query
def execute_sql_query(db_file, sql_query):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()
        return pd.DataFrame(result, columns=columns)
    except sqlite3.Error as e:
        conn.close()
        return f"SQL error: {e}"
    
# Generate Natural Language Response using LLM
def generate_response(llm, dataframe, standalone_question, chat_history):
    """
    Generate a concise natural language response based on the standalone question,
    the SQL query result, and the chat history. Incorporate special instructions for uniform interpretation of key terms.
    """
    special_instructions = """
Interpret the following terms consistently:
- "Summarize," "Describe," and "Explain" all mean "Provide a detailed breakdown with comprehensive coverage of the topic."
- Focus on clarity, detail, and coherence in the response.
- Avoid generic phrases or filler text (e.g., "Based on the information provided").
- Do not assume missing or unrelated information unless explicitly stated in the data.
"""
    
    if isinstance(dataframe, str) and "SQL error" in dataframe:
    # Handle SQL syntax errors
        error_prompt = f"""
    The following request could not be processed due to a technical error in generating or executing the SQL query.
    Explain this to the user in simple terms without using technical jargon like 'SQL error' or 'query execution.'
    Focus on a helpful and user-friendly response.

    User Request: {standalone_question}

    Chat History:
    {chat_history}

    Error Details:
    {dataframe}
    """
        response = llm.invoke(error_prompt)
        return response.content.strip()

    elif dataframe.empty:   
    # Handle empty results
        no_data_prompt = f"""
    The following request was made, but the system could not find any relevant information. 
    Provide a concise and direct response to the user, following these special instructions:

    {special_instructions}

    User Request: {standalone_question}

    Chat History:
    {chat_history}
    """
        response = llm.invoke(no_data_prompt)
        return response.content.strip()

    else:
        # Convert the DataFrame to a comma-separated format including column names
        result_text = ", ".join(dataframe.columns) + "\n"  # Add column names
        result_text += "\n".join(dataframe.astype(str).apply(lambda row: ", ".join(row), axis=1).tolist())

        # Use the LLM to generate a response based on the query result and chat history
        response_prompt = f"""
Provide a response based on the retrieved information. Follow these special instructions:

{special_instructions}

User Request: {standalone_question}

Information Retrieved:
{result_text}

Chat History:
{chat_history}
"""
        response = llm.invoke(response_prompt)
        return response.content.strip()



### Q&A 

In [156]:
# Initialize the ConversationManager with a history limit of 5
conv_manager = ConversationManager(history_limit=3)


In [157]:
# Step 1: Generate SQL Query
user_question = "Show me top 10 claim numbers where the loss cause is glass break."
standalone_question = conv_manager.reformulate_question(user_question, llm)
print(f"Standalone Question: {standalone_question}")
sql_query = generate_sql_query(llm, standalone_question, table_metadata, table_name, table_schema)

# Step 2: Execute SQL Query with Preprocessing
query_result = execute_sql_query_with_preprocessing(db_file, sql_query, faiss_store, embedding_model)

# Step 3: Generate Response
response = generate_response(llm, query_result, standalone_question, conv_manager.get_conversation_context())
print(response)
conv_manager.update_conversation_history(standalone_question, query_response)


Standalone Question: What are the top 10 claim numbers with a loss cause of glass break?
Original SQL query: SELECT claim_no 
FROM claims 
WHERE loss_cause LIKE '%glass break%' 
ORDER BY claim_no 
LIMIT 10;
Standardized loss_cause: Glass breakage
standard SQL query:SELECT claim_no 
FROM claims 
WHERE loss_cause LIKE '%glass break%' 
ORDER BY claim_no 
LIMIT 10;
Here are the top 10 claim numbers with a loss cause of glass break:
- CA0000007
- CA0000009
- CA0000010
- CA0000011
- CA0000012
- CA0000013
- CA0000018
- CA0000020
- CA0000033
- CA0000035

These are the specific claim numbers that have been reported with a loss cause of glass break.


In [148]:
sql_query

"SELECT claim_no \nFROM claims \nWHERE loss_cause LIKE '%glass break%' \nORDER BY claim_no \nLIMIT 10;"

In [ ]:
if "loss_cause" in sql_query.lower():
        # Extract the loss_cause value from the WHERE clause
        import re
        match = re.search(r"loss_cause\s*(ILIKE|LIKE|=)\s*'%([^']+)%?'", sql_query, re.IGNORECASE)
        if match:
            loss_cause_value = match.group(2)
            print(loss_cause_value)
            # Search FAISS for the closest match
            query_embedding = embedding_model.embed_query(loss_cause_value)
            matches = faiss_store.similarity_search_by_vector(query_embedding, k=1)

            if matches:
                closest_match = matches[0].page_content
                print(closest_match)
                # Replace the original value with the standardized value
                sql_query = sql_query.replace(f"'{loss_cause_value}'", f"'{closest_match}'")
                print(f"Standardized loss_cause: {closest_match}")

glass break%
Glass breakage
Standardized loss_cause: Glass breakage


In [150]:
match

In [87]:
# Initialize the ConversationManager with a history limit of 5
conv_manager = ConversationManager(history_limit=3)


In [88]:
# Step 3: Get user query and convert it
user_question = "Summarize the FNOL call for Claim number CH0000002 in detail"

# Reformulate the question (if needed)

standalone_question = conv_manager.reformulate_question(user_question, llm)
print(f"Standalone Question: {standalone_question}")

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query_with_preprocessing(db_file, sql_query, faiss_store, embedding_model)
response = generate_response(llm, query_result, standalone_question, conv_manager.get_conversation_context())
print(response)
conv_manager.update_conversation_history(standalone_question, query_response)

Standalone Question: What are the detailed steps involved in summarizing the FNOL call for Claim number CH0000002?
Generated SQL Query: SELECT fnol_call, adjuster_notes
FROM claims
WHERE claim_no = 'CH0000002'
To summarize the FNOL call for Claim number CH0000002, the customer, Sarah Johnson, reported a property loss at her home in Springfield, Illinois on June 5th, 2021. The incident was related to the removal of a large tree that fell onto her house during a storm, causing extensive damage that required immediate attention. The tree removal process cost Sarah around $1,000, and the damage to her roof and siding was also significant, but fortunately, her insurance covered most of it. Sarah expressed gratitude that no one was hurt during the incident, and the situation has been resolved satisfactorily according to her. The adjuster's notes indicate that a short form bill for tree removal was paid for $1,000.00.


In [89]:
conv_manager.get_conversation_context()

['Question: What are the detailed steps involved in summarizing the FNOL call for Claim number CH0000002? \n Answer: The policy number associated with claim number CH0000028 is H00000459.']

In [90]:
# Step 3: Get user query and convert it
user_question = "Summarize the Adjustor notes for claim number CH0000002"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

print(f"Standalone Question: {standalone_question}")

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query_with_preprocessing(db_file, sql_query, faiss_store, embedding_model)
response = generate_response(llm, query_result, standalone_question, conv_manager.get_conversation_context())
print(response)
conv_manager.update_conversation_history(standalone_question, query_response)

Standalone Question: What are the steps to summarize the Adjustor notes for claim number CH0000002?
Generated SQL Query: SELECT adjuster_notes 
FROM claims 
WHERE claim_no = 'CH0000002'
To summarize the Adjustor notes for claim number CH0000002, we need to provide a detailed breakdown of the incident and its aftermath. On June 5th, 2021, the customer reported a property loss at their home in Springfield, Illinois, due to a large tree falling onto their house during a storm. The damage caused by the fallen tree was extensive and required immediate attention. However, the situation has been resolved satisfactorily according to the customer. The tree removal process cost the customer around $1,000, and the damage to the roof and siding was also significant, but most of it was covered by insurance. The customer expressed gratitude that no one was hurt during the incident. Additionally, a short form bill for tree removal was paid for $1,000.00.


In [91]:
# Step 3: Get user query and convert it
user_question = "What is the policy Number of claim number: CH0000028?"

# Reformulate the question (if needed)

standalone_question = conv_manager.reformulate_question(user_question, llm)
print(f"Standalone Question: {standalone_question}")

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
         # Print the response line by line
        print("LLM Response:")
        for line in query_response.splitlines():
                print(line)
conv_manager.update_conversation_history(standalone_question, query_response)

Standalone Question: What is the policy number associated with claim number CH0000028?
Generated SQL Query: SELECT PolicyNumber 
FROM claims 
WHERE claim_no = 'CH0000028'
LLM Response:
The policy number associated with claim number CH0000028 is H00000459.


In [94]:
conv_manager.clear_history()

In [95]:
# Step 3: Get user query and convert it
user_question = "Give me the top 10 Claims numbers for cases with Loss cause like Glass breakage"

# Reformulate the question (if needed)

standalone_question = conv_manager.reformulate_question(user_question, llm)
print(f"Standalone Question: {standalone_question}")
sql_query = generate_sql_query(llm, standalone_question, table_metadata, table_name, table_schema)

# Step 2: Execute SQL Query with Preprocessing
query_result = execute_sql_query_with_preprocessing(db_file, sql_query, faiss_store, embedding_model)

# Step 3: Generate Response
response = generate_response(llm, query_result, standalone_question, conv_manager.get_conversation_context())
print(response)
conv_manager.update_conversation_history(standalone_question, query_response)

Standalone Question: What are the top 10 Claims numbers for cases with a Loss cause of Glass breakage?
Standardized loss_cause: Glass breakage
Here are the top 10 Claims numbers for cases with a Loss cause of Glass breakage:
- CA0000007
- CA0000009
- CA0000010
- CA0000011
- CA0000012
- CA0000013
- CA0000018
- CA0000020
- CA0000033
- CA0000035

These are the specific claim numbers associated with glass breakage losses, ranked in order from highest to lowest.


In [313]:
# Step 3: Get user query and convert it
user_question = "Summarize all of her Clinical Notes"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE Nurse_name LIKE "%Olivia%"
Query Result:
                                      Clinical_Notes
0  Nurse_Note: Primary CNA was undressing residen...
1  Nurse_Note: Safely You called to apprise that ...
2  Nurse_Note: During care resident's aide observ...
Nurse Olivia's clinical notes contain three entries. The first entry describes an incident where the primary CNA was undressing a resident in bed and accidentally caused a skin tear on the left lower leg. The second entry reports an incident where the Elder was looking for her watch and moving around the room, but did not fall. The third entry notes that a resident had open skin on their right lower extremity due to dry skin and discoloration.


In [254]:
# Step 3: Get user query and convert it
user_question = "Were there any injuries in any of her notes"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE Nurse_name LIKE "%Olivia%" AND Clinical_Notes LIKE "%injuries%"
Query Result:
Empty DataFrame
Columns: [Clinical_Notes]
Index: []
Yes, Nurse Olivia's clinical notes mention three injuries. The first injury was a skin tear on the left lower leg caused by the primary CNA while undressing a resident in bed. The second injury was a resident moving towards the floor while looking for her watch, but not falling. The third injury was open skin on the right lower extremity with surrounding skin discoloration and dry skin, and the resident was unable to explain how she got the injury due to cognitive deficit.


In [298]:
conv_manager.clear_history()

In [299]:
# Step 3: Get user query and convert it
user_question = "Summarize Notes for Robert Nash, only Clinical"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE First_Name = 'Robert' AND Last_Name = 'Nash'
Query Result:
                                      Clinical_Notes
0  Nurse_Note: Nurse responded to a call light in...
Based on the information retrieved, it appears that Robert Nash fell while sitting on the floor next to his bed. He was found by a nurse who responded to his call light. He was last seen sitting in his recliner chair about an hour before the fall and denied being in pain. His vital signs were normal, and he did not appear to have any injuries. His wife was informed, and their son was called about the fall. Dr. Kobylarz was also made aware of the situation.


In [300]:
conv_manager.clear_history()

In [301]:
# Step 3: Get user query and convert it
user_question = "Summarize Notes where skin tear was observed"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes, Non_clinical_notes 
FROM nurse_notes 
WHERE Clinical_Notes LIKE "%skin tear%" OR Non_clinical_notes LIKE "%skin tear%"
Query Result:
                                      Clinical_Notes  \
0  Nurse_Note: Primary CNA was undressing residen...   
1  Nurse_Note: 5:50pm I was in room 35 assisting ...   

                                  Non_clinical_notes  
0  Nurse_Note: At 11:00 am on 09/20/2023 nurse wa...  
1  Nurse_Note: At 5:30 pm on 02/05/2024 nurse was...  
Based on the notes retrieved, there were three instances of observed skin tears. The first one happened when the primary CNA was undressing Resident in bed and accidentally caused a skin tear on the left lower leg front area. The second one was observed on the left lateral aspect of Resident's knee and left lateral elbow after Resident fell and was transferred to RWJUH. The third one was not specified where it occurred.


In [302]:
# Step 3: Get user query and convert it
user_question = "what were the names of the residents with skin tear"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT First_Name, Last_Name, Clinical_Notes 
FROM nurse_notes 
WHERE Clinical_Notes LIKE "%skin tear%"
Query Result:
  First_Name Last_Name                                     Clinical_Notes
0      James    Miller  Nurse_Note: Primary CNA was undressing residen...
1     Olivia  Martinez  Nurse_Note: 5:50pm I was in room 35 assisting ...
Based on the information retrieved, James Miller experienced a skin tear on his left lower leg front area while being undressed by a CNA. Olivia Martinez experienced skin tears on her left lateral aspect of her knee and left lateral elbow after falling next to the nursing station.


In [303]:
conv_manager.clear_history()
conv_manager.history

[]

In [304]:
# Step 3: Get user query and convert it
user_question = "Summarize any fall incidents, Name the residents as well"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT First_Name, Last_Name, Clinical_Notes 
FROM nurse_notes 
WHERE Clinical_Notes LIKE '%fall%'
Query Result:
  First_Name Last_Name                                     Clinical_Notes
0      James    Miller  Nurse_Note: Approximately 7:30 pm Nurse respon...
1    Michael   Johnson  Nurse_Note: Safely You called to apprise that ...
2    Jessica     Brown  Nurse_Note: It was reported to me by the Nurse...
3     Robert      Nash  Nurse_Note: Nurse responded to a call light in...
Yes, I can provide you with a summary of falls that have occurred and the names of the residents involved. James Miller fell and was assisted to transfer via mechanical lift. Michael Johnson did not fall but was observed visually and tactilely scanning the area for her "watch." Jessica Brown fell while reaching for items in the cabinet and was assisted to a wheelchair by two staff for safety. Robert Nash fell and was assisted into bed at his request.


In [307]:
conv_manager.clear_history()
conv_manager.history

[]

In [239]:
# Step 3: Get user query and convert it
user_question = "Summarize non clinical notes for Resident Robert Nash"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Non_clinical_notes FROM nurse_notes WHERE First_Name = 'Robert' AND Last_Name = 'Nash'
Query Result:
                                  Non_clinical_notes
0  Nurse_Note: At 3:00 pm on 12/15/2023 nurse was...
Based on the information retrieved, it looks like there is a non-clinical note for Resident Robert Nash that needs to be summarized. The note is from a nurse who was in the recreation room on December 15th, 2023. The note mentions that an elder in Room #11A reported a lack of activities and was found sitting alone, expressing boredom and a desire for more engaging activities. The nurse discussed this with the elder and the activities coordinator was notified to include the elder's preferences in planning. The elder was invited to participate in an upcoming activity and seemed appreciative.


In [308]:
# Step 3: Get user query and convert it
user_question = "explain clinical notes for each female in the data"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE Gender = 'Female'
Query Result:
                                      Clinical_Notes
0  Nurse_Note: At 3:10 am on 07/17/2021 writer wa...
1  Nurse_Note: Resident's pendant went off at 5:2...
2  Nurse_Note: It was reported to me by the Nurse...
3  Nurse_Note: 5:50pm I was in room 35 assisting ...
4  Nurse_Note: Resident's pendant went off at 5:2...
The clinical notes provide information about incidents that occurred with female residents. For example, one resident fell out of bed and was found sitting on the floor, another resident fell while reaching for items in a cabinet, and another resident had loose bowel movements. The notes describe the actions taken by the nursing staff, such as assessing the residents for injuries, providing assistance, and contacting physicians or family members. The notes also include vital signs and any medication administered.
